# Чат-бот с функциями (продуктовый помощник)
Source: [chat_bot_with_functions.ts](source/chat_bot_with_functions.ts)

В этом примере разбирается как создается простой бот, который может обращаться к внешним инструментам.

Здесь мы создадим бота который:
* Может получать какие продукты есть в магазине
* Получать детальную информацию по конкретной позиции

In [ ]:
import GigaChat from 'gigachat';
import * as dotenv from 'dotenv';
import { Agent } from 'node:https';

dotenv.config();

const httpsAgent = new Agent({
    rejectUnauthorized: false,
});

const client = new GigaChat({
    timeout: 600,
    model: 'GigaChat-Max',
    httpsAgent: httpsAgent,
});

In [4]:
import { Message } from 'gigachat/interfaces';
const messages: Message[] = [
  {
    role: 'system',
    content: `Ты бот-консультант продуктового магазина. 
  Помни, что у тебя есть следующие функции:
    get_products: Получает текущие позиции в магазине
    get_product_info: Получает информацию о продукте (цена, количество)
  Помни, что ты можешь вызывать функции последовательно
  `,
  },
];

Объявление базы данных

Здесь мы создаем фейковые продукты, но также можно реализовать доступ к внешней БД

In [5]:
const PRODUCTS_DB = [
    {
        name: 'Молоко',
        price: 100,
        quantity: 10,
        id: 0,
    },
    {
        name: 'Хлеб',
        price: 70,
        quantity: 0,
        id: 1,
    },
    {
        name: 'Сыр',
        price: 200,
        quantity: 20,
        id: 3,
    },
    {
        name: 'Масло',
        price: 170,
        quantity: 20,
        id: 4,
    },
];

Объявляем внешние инструменты

In [6]:
function get_products(): any {
  console.log('Вызов функции: get_products');
  return PRODUCTS_DB.map((product) => {
    return { name: product.name, id: product.id };
  });
}

function get_product_info(args: any) {
  console.log('Вызов функции: get_products_info');
  console.log(`С параметрами: ${JSON.stringify(args)}`);
  return PRODUCTS_DB.filter((product) => args.ids.includes(product.id));
}

const functions = [
  {
    name: 'get_products',
    description: 'Получает какие продукты есть в магазине',
    parameters: {
      type: 'object',
      properties: {
        arg: {
          type: 'string',
        },
      },
    },
    return_parameters: {
      type: 'object',
      properties: {
        name: {
          type: 'string',
          description: 'Название позиции',
        },
        id: {
          type: 'string',
          description: 'ID позиции',
        },
      },
    },
  },
  {
    name: 'get_product_info',
    description: 'Получает информацию об определенном продукте в магазине',
    parameters: {
      type: 'object',
      properties: {
        ids: {
          type: 'array',
          items: {
            type: 'number',
            description: 'ID продуктов',
          },
        },
      },
    },
    return_parameters: {
      type: 'object',
      properties: {
        products: {
          type: 'array',
          items: {
            type: 'object',
            properties: {
              name: {
                type: 'string',
                description: 'Название позиции',
              },
              id: {
                type: 'string',
                description: 'ID позиции',
              },
              quantity: {
                type: 'string',
                description: 'Количество',
              },
              price: {
                type: 'string',
                description: 'Цена',
              },
            },
          },
        },
      },
    },
  },
]

In [9]:
let question = prompt('Q: ');
while (question) {
    console.log(`Q: ${question}`)
    messages.push({
        role: 'user',
        content: question,
    });
    let response = await client.chat({
        functions: functions,
        messages,
        function_call: "auto"
    });
    if (response.choices[0]?.message) messages.push(response.choices[0].message);
    if (response.choices[0]?.message.function_call) {
        let function_result: any = {};
        switch (response.choices[0].message.function_call.name) {
            case 'get_products':
                function_result = get_products();
                break;
            case 'get_product_info':
                function_result = get_product_info(response.choices[0].message.function_call.arguments);
                break;
        }
        messages.push({
            role: 'function',
            content: JSON.stringify(function_result),
        });
        response = await client.chat({
            functions,
            messages,
        });
    }
    console.log(`AI: ${response.choices[0]?.message.content}`)
    question = prompt('Q: ');
}

Q: Какие продукты есть?
Вызов функции: get_products
AI: В нашем магазине доступны следующие продукты: молоко, хлеб, сыр, масло. Какой из них вас интересует?
Q: Сколько будет сделать бутерброд с сыром?
Вызов функции: get_products_info
С параметрами: {"ids":[1,3]}
AI: Чтобы приготовить бутерброд с сыром, потребуется один кусок хлеба и немного сыра. Хлеб стоит 70 рублей за штуку, а сыр - 200 рублей за килограмм. Поскольку для одного бутерброда нужно небольшое количество сыра, предположим, что это примерно 50 граммов, что составит около 10 рублей. Итак, общая стоимость бутерброда с сыром будет приблизительно 80 рублей.
Q: А если туда добавить масло?
Вызов функции: get_products_info
С параметрами: {"ids":[4]}
AI: Если добавить масло, то стоимость бутерброда увеличится. Давайте посчитаем общую сумму.

Стоимость хлеба - 70 рублей.
Стоимость сыра - 200 рублей.
Стоимость масла - 170 рублей.

Общая сумма за бутерброд с сыром и маслом составит 440 рублей.


""